In [12]:
import lightning.pytorch as pl
import torch
from transformers import AutoProcessor, Qwen2VLForConditionalGeneration, Qwen2_5_VLForConditionalGeneration
from qwen_vl_utils import process_vision_info
import torchvision.transforms as T
from PIL import Image
import json
import re


def clean_json_str(s):
    # 코드블록 마크다운 제거
    s = re.sub(r"^```", "", s.strip(), flags=re.MULTILINE)
    return s

# default processer
model_name = "Qwen/Qwen2.5-VL-3B-Instruct" #Qwen/Qwen2.5-VL-3B-Instruct, Qwen/Qwen2-VL-2B-Instruct, Qwen/Qwen2.5-VL-7B-Instruct
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(model_name, torch_dtype="auto", device_map="auto")

processor = AutoProcessor.from_pretrained(model_name)

# The default range for the number of visual tokens per image in the model is 4-16384.
# You can set min_pixels and max_pixels according to your needs, such as a token range of 256-1280, to balance performance and cost.
# min_pixels = 256*28*28
# max_pixels = 1280*28*28
# processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct", min_pixels=min_pixels, max_pixels=max_pixels)
pil_image1 = Image.open("/data/ephemeral/home/work/python/gx-ocr/data/datasets/images/test/drp.en_ko.in_house.selectstar_000002.jpg")
pil_image2 = Image.open("/data/ephemeral/home/work/python/gx-ocr/data/datasets/images/test/drp.en_ko.in_house.selectstar_000013.jpg")

text = "Detect all text in the image and return their locations in the form of coordinates.  ex) [{'bbox_2d': [106, 345, 178, 496]}, ...]"

messages = [

    #[{"role": "user", "content": [{"type": "image", "image": pil_image1}, {"type": "text", "text": text}]}],
    [{"role": "user", "content": [{"type": "image", "image": pil_image2}, {"type": "text", "text": text}]}],

]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

#print(text)


image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=text,
    images=image_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=4096)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
#print(clean_json_str(output_text[0]))
print(output_text)

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.54it/s]


['```json\n[\n\t{"bbox_2d": [106, 345, 178, 496], "text_content": "EGG DROP"},\n\t{"bbox_2d": [106, 500, 178, 651], "text_content": "주문번호 : 0015"},\n\t{"bbox_2d": [106, 654, 178, 704], "text_content": "점포명 : 에그드랍 동성로중앙점"},\n\t{"bbox_2d": [106, 705, 178, 755], "text_content": "주소 : 대구중구 동성로1길 29-35"},\n\t{"bbox_2d": [106, 756, 178, 806], "text_content": "사업자번호 : 891-21-00828"},\n\t{"bbox_2d": [106, 807, 178, 857], "text_content": "대표자명 :"},\n\t{"bbox_2d": [106, 858, 178, 908], "text_content": "전화번호 :"},\n\t{"bbox_2d": [106, 909, 178, 959], "text_content": "[신용카드승인]"},\n\t{"bbox_2d": [106, 960, 178, 1010], "text_content": "카드종류 : 시티카드"},\n\t{"bbox_2d": [106, 1011, 178, 1061], "text_content": "카드번호 : 524144***********"},\n\t{"bbox_2d": [106, 1062, 178, 1112], "text_content": "승인번호 : 76764996"},\n\t{"bbox_2d": [106, 1113, 178, 1163], "text_content": "할부기간 : 일시불"},\n\t{"bbox_2d": [106, 1164, 178, 1214], "text_content": "이름 :"},\n\t{"bbox_2d": [106, 1215, 178, 1265], "text_content": "개수 :"},